In [1]:
import spacy
from spacy.lang.en import English
from spacy.lang.es import Spanish
from spacy.pipeline import EntityRuler
import json
import random
from spacy.tokens import Doc
from spacy.training import Example
from spacy.language import Language

/Users/dianitalopez/Documents/GitHub/Corridos/.p3venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#don't mess with the codeblock box
#1 NER Spacy create training data
import json 

def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        print(f)
        return(json.load(f))
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       data = json.dump(data, f, indent=4)
       print(data)


In [3]:
#dont mess with the codeblock box 
# 1 NER Spacy create training data

def create_training_data(file, type):
    #print(type) #gpe
    #print(file) # TexasNER_GPE_master
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        print(key)
        if key == "List of Municipalities":
            for obj in value: #muni loop
                #print(obj.get("Municipality"))
                pattern = {
                    "label":type,
                    "pattern": obj.get("Municipality")
                    }
                patterns.append(pattern) # end of muni loop
            for obj in value: #primary county loop 
                #if any(d.get("Primary County") == "Gonzales" for d in value): 
                    #print("found in Municipality") 
                if not any(d.get("Municipality") == obj.get("Primary County") for d in value):
                    #print('not found in Manicipality')
                    pattern = {
                        "label":type,
                        "pattern": obj.get("Primary County")
                        }
                    patterns.append(pattern)           

        if key == "List of Unincorporated Communities":
            for obj in value: #unicorporated loop
                print(obj.get("Unincorporated Community"))
                pattern = {
                    "label":type,
                    "pattern": obj.get("Unincorporated Community")
                    }
                patterns.append(pattern) # end of community name loop
            for obj in value: #county loop 
                if not any(d.get("Unincorporated Community") == obj.get("Primary County") for d in value):
                    pattern = {
                        "label":type,
                        "pattern": obj.get("Primary County")
                        }
                    patterns.append(pattern)                 

        if key == "List of Ghost Town":
            for obj in value: #ghost town loop
                #print(obj.get("Ghost town"))
                pattern = {
                    "label":type,
                    "pattern": obj.get("Ghost town")
                    }
                patterns.append(pattern) # end of muni loop
            for obj in value: #county loop 
                if not any(d.get("Ghost town") == obj.get("Primary County") for d in value):
                    pattern = {
                        "label":type,
                        "pattern": obj.get("Primary County")
                        }
                    patterns.append(pattern)

        #for obj in value: # unicorpated communities loop
                #if any(d.get("Primary County")=='Karnes' for d in value): 
                    #print("found in Primary County")
        
            #print(pattern)
            #print(patterns)

    return(patterns)
patterns = create_training_data('../Corridos/data/TexasNER_GPE_master.json', 'GPE')
#print(patterns)

<_io.TextIOWrapper name='../Corridos/data/TexasNER_GPE_master.json' mode='r' encoding='utf-8'>
List of Municipalities
List of Unincorporated Communities
Alderbranch
Bethel
Blackfoot
Bois d'Arc
Bradford
Broom City
Brushy Creek
Cedar Creek
Cayuga
Cedar Creek
Cronin
Crystal Lake
Days Chapel
Denson Springs
Elmtown
Elmwood
Fosterville
Greens Bluff
Long Lake
Massey Lake
Montalba
Mound City
Myrtle Springs
Neches
Pert
Providence
Redtown
Salmon
Slocum
Springfield
Swanson Hill Church
Tennessee Colony
Todd City
Tucker
Wells Creek
Yard
Florey
Frankel City
McKinney Acres
Shafter Lake
Herty
Homer
Oak Flat
Pollok
Redland
Holiday Beach
Lamar
Dundee
Huff
Mankins
Washburn
Goodnight
Wayside
Amphion
Campbellton
Leming
Peggy
McCoy
Bleiblerville
Cat Spring
Kenney
New Ulm
Shelby
Bula
Enochs
Virginia City
Maple
Needmore
Bandera Falls
Lake Medina Shores
Lakehills
Pipe Creek
Tarpley
Vanderpool
Medina
Alum Creek
Bateman
Butler
Camp Swift
Cedar Creek
Circle D-KC Estates
Colorado
Jeddo
Jorden
Kovar
McDade
Paige
Pe

In [4]:
# start run here iff NEEDED. 
# don't mess with this codeblock box
#TX NER TRAINING SET 
#from 2 NER Spacy- Creating NER training data set Notebook

def generate_rules(patterns):
    nlp = English()
    #ruler = EntityRuler(nlp)
    ruler = nlp.add_pipe('entity_ruler', config={"validate": True}) 
    ruler.add_patterns(patterns) 
    nlp.to_disk("tx_trained_ner")

def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) >0: #if entity has been found move entity to results and entity list
        results = [text, {"entities": entities}]
        #print(results)
    return (results)

#patterns = create_training_data('../Corridos/data/TexasNER_GPE_master.json', 'GPE')
generate_rules(patterns)
#print(patterns)

#how spacy wants to see read the data#
#TRAIN_DATA = [(text, {"entities": [(start, end, label)]})]#

nlp = spacy.load("tx_trained_ner")
TRAIN_DATA= []

with open ("../Corridos/corrido corpus/ElCorridodeGregorioCortez_X.txt", "r", encoding='utf-8')as f:
    text = f.read()
    #print(text)
    segments = text.split("\n\n")[0:]
    #cleaning up the lyric text. making it easier to read for the program
    for segment in segments:
        segment = segment.strip() #might not need this code line 
        segment = segment.replace("\n", " ")# might not need this code line
        #print(segment)
        punc = '[":;,“.”[@_!$%^&*()<>?/\|}{~:]#]'
        for ele in segment:
            if ele in punc:
                segment = segment.replace(ele, "")

        #print(segment)

        results = test_model(nlp, segment)
        if results != []: #it found something and return it
            TRAIN_DATA.append(results)
        print(results)

#print(TRAIN_DATA)
save_data("../Corridos/data/Tx_NER_GPE_training_data.json", TRAIN_DATA)

['In the country of Karnes  Look what has happened  The Major Sheriff died Leaving Román badly wounded', {'entities': [(18, 24, 'GPE')]}]
[]
[]
[]
[]
[]
[]
[]
[]
['He struck out for Gonzales Without showing any fear Follow me cowardly rangers  I am Gregorio Cortez', {'entities': [(18, 26, 'GPE')]}]
['From Belmont he went to the  ranch   They succeeded in surrounding  him  Quite a few more than three hundred  But there he jumped their corral', {'entities': [(5, 12, 'GPE')]}]
[]
[]
['Gregorio Cortez went out  He went towards Laredo They decided not to follow  Because they were afraid of him', {'entities': [(42, 48, 'GPE')]}]
[]
[]
['Over by El Encinal  According to what we hear  They made him a corral  And he killed them another sheriff', {'entities': [(11, 18, 'GPE')]}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
None


In [5]:
def train_spacy(data, iterations): #passing training data/set and the number/generation of the training process. 30 is good. long time!!!!
    #print(data)
    TRAIN_DATA = data
    #print(TRAIN_DATA)
    nlp = spacy.blank("en")
    #print(nlp)
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner") #if there is no ner in the pipe, create a pipe
        nlp.add_pipe("ner", last=True)
    
    for _, annotations in TRAIN_DATA: #add labels
        print(annotations.get("entities"))
        for ent in annotations.get("entities"):
            ner.add_label(ent[2]) # i'm only working with one label 'GPE' if more increase i believe double check
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]  
    with nlp.disable_pipes(*other_pipes): #won't mess up other pipes
        optimizer = nlp.begin_training()
        for itn in range(iterations): # adjust the funtion not the items inside. arg of interations 30 is good
            print("Starting iteration "+ str(itn)) #where I am at in the interations process
            random.shuffle(TRAIN_DATA) #shuffle helps the program to learn not memorize order. 
            losses = {}
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations) #fix found here for ValueError: [E973] Unexpected type for NER data https://github.com/explosion/spaCy/issues/7038
                nlp.update([example], drop = 0.2, sgd= optimizer, losses=losses)            
            print(losses)  
    
    return(nlp) #model return

#TRAIN_DATA = load_data("../Corridos/data/TxMunicipality_training_data.json")
#print(TRAIN_DATA)
nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("tx_ner_model")


[(18, 24, 'GPE')]
[(18, 26, 'GPE')]
[(5, 12, 'GPE')]
[(42, 48, 'GPE')]
[(11, 18, 'GPE')]
Starting iteration 0
{'ner': 83.0798956155777}
Starting iteration 1
{'ner': 49.846851371228695}
Starting iteration 2
{'ner': 11.822019466868369}
Starting iteration 3
{'ner': 9.44329464098568}
Starting iteration 4
{'ner': 6.895927057358646}
Starting iteration 5
{'ner': 4.494940846266912}
Starting iteration 6
{'ner': 2.9394570848749786}
Starting iteration 7
{'ner': 1.1887205828342402}
Starting iteration 8
{'ner': 0.05216680553363832}
Starting iteration 9
{'ner': 0.0007934535603407197}
Starting iteration 10
{'ner': 0.00017801535653927736}
Starting iteration 11
{'ner': 2.502979575364497e-05}
Starting iteration 12
{'ner': 3.429139811213685e-07}
Starting iteration 13
{'ner': 5.2497348541658624e-09}
Starting iteration 14
{'ner': 1.5141124595044897e-07}
Starting iteration 15
{'ner': 2.572725366484365e-09}
Starting iteration 16
{'ner': 1.1810687889970429e-08}
Starting iteration 17
{'ner': 1.6801211615262016

In [7]:
#time to test
test= "corrido corpus/ElCorridodeGregorioCortez_X.txt"
with open(test, 'r', encoding='utf-8') as c:
    test = c.read()


import re

def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
test = clean_text(test)
print(test)

TxGPE=[]
nlp = spacy.load("tx_ner_model")
doc =nlp(test) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)
'''   
nlp = spacy.load("en_core_web_lg") 
doc = nlp(test)
TxGPE=[]
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)
'''

In the country of Karnes 
Look what has happened 
The Major Sheriff died
Leaving Román badly wounded

It must have been two in the 
afternoon
When people arrived
They said to one another
It is not known who killed him

They went around asking questions
About half an hour afterward 
They found that the wrongdoer
Had been Gregorio Cortez

Now they have outlawed Cortez
Throughout the whole state
Let him be taken dead or alive 
He has killed several men

They said Gregorio Cortez 
With his pistol in his hand
I don’t regret that I killed him 
I regret my brother’s death

Then said Gregorio Cortez 
And his soul was all aflame 
I don’t regret that I killed him
A man must defend himself

The Americans were coming 
They were whiter than a dove 
From the fear that they had 
Of Cortez and of his pistol 

Then the Americans said 
Then they said fearfully
Come let us follow the trail
The wrongdoer is Cortez

They set the bloodhounds on him 
So they could follow his trail 
But trying to overtake  Co

'   \nnlp = spacy.load("en_core_web_lg") \ndoc = nlp(test)\nTxGPE=[]\nfor ent in doc.ents:\n    #print(ent.text, ent.label_)\n    if ent.label_ == "GPE":\n        TxGPE.append(ent.text)\nprint(TxGPE)\n'